## การเตรียมชุดข้อมูลตัวอย่างของแอปเปิ้ลสายพันธุ๋ Envi ใน UK

- ทำการอ่านไฟล์ Apple462.csv 
- คัดกรองชุดข้อมูลตัวอย่างเฉพาะประเทศ UK

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('../Apple462.csv')
df = df.drop(df[df["region"] != "UK"].index)
df = df.drop(columns=["Unnamed: 0"])
df.head(10)

,Date,Envi,Fuji,Gala,region
156,27/12/2015,30482,2972,5894,UK
157,20/12/2015,25628,2546,7394,UK
158,13/12/2015,27149,2986,6458,UK
159,6/12/2015,35591,12527,4086,UK
160,29/11/2015,24190,2790,6031,UK
161,22/11/2015,,3256,6663,UK
162,15/11/2015,24425,12086,6502,UK
163,8/11/2015,31910,4652,6286,UK
164,1/11/2015,30435,4360,6428,UK
165,25/10/2015,29522,10090,6552,UK


- นำ features ของวันที่และสายพันธุ์ Envi แล้วสร้าง
ชุดข้อมูลตัวอย่างใหม่

In [2]:
df_envi = df.drop(columns=["Fuji", "Gala", "region"])
df_envi.head(10)

,Date,Envi
156,27/12/2015,30482
157,20/12/2015,25628
158,13/12/2015,27149
159,6/12/2015,35591
160,29/11/2015,24190
161,22/11/2015,
162,15/11/2015,24425
163,8/11/2015,31910
164,1/11/2015,30435
165,25/10/2015,29522


- เปลี่ยนชนิดข้อมูลของวันที่จาก object เป็น date
- นำชุดข้อมูลตัวอย่างมาเรียงชุดข้อมูลตัวอย่างใหม่ตามวันที่จากเก่าไปใหม่

In [3]:
df_envi["Date"] = pd.to_datetime(df_envi["Date"], format="%d/%m/%Y")
df_envi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169 entries, 156 to 3344
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    169 non-null    datetime64[ns]
 1   Envi    169 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 4.0+ KB


In [4]:
df_envi.sort_values(by="Date", inplace=True)
df_envi["Date"] = df_envi["Date"].dt.strftime("%d/%m/%Y")
df_envi.head(10)

,Date,Envi
207,04/01/2015,44562
206,11/01/2015,38112
205,18/01/2015,50355
204,25/01/2015,51922
203,01/02/2015,67496
202,08/02/2015,54229
201,15/02/2015,41548
200,22/02/2015,50872
199,01/03/2015,40578
198,08/03/2015,


- ตรวจสอลชุดข้อมูลตัวอย่างว่ามี miss value ในวันที่ใดบ้าง

In [5]:
result = df_envi[df_envi['Envi'] == ' ']
result

,Date,Envi
198,08/03/2015,
182,28/06/2015,
161,22/11/2015,
1250,11/12/2016,
2387,26/02/2017,
2345,17/12/2017,
3340,04/02/2018,


- แก้ไข miss value โดยหาค่าเฉลี่ยของ 2 วันก่อนหน้า
- หลังจากแก้ไขแล้วให้เปลี่ยนชนิดข้อมูลของชุดข้อมูลตัวอย่างของ Envi ให้เป็นจำนวนเต็ม ( Integer ) เนื่องจากค่าเฉลี่ยทีได้อาจได้ค่าเป็นทศนิยม ( Float )

In [6]:
for x in df_envi.index:
    if df_envi.loc[x, "Envi"] == " ":
        df_envi.loc[x, "Envi"] = np.nan
df_envi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169 entries, 207 to 3333
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    169 non-null    object
 1   Envi    162 non-null    object
dtypes: object(2)
memory usage: 8.0+ KB


In [7]:
df_envi["Envi"] = pd.to_numeric(df_envi["Envi"])
df_envi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169 entries, 207 to 3333
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    169 non-null    object 
 1   Envi    162 non-null    float64
dtypes: float64(1), object(1)
memory usage: 8.0+ KB


In [8]:
df_envi = df_envi.interpolate()
df_envi["Envi"] = df_envi["Envi"].astype("int64")
df_envi.head(10)

,Date,Envi
207,04/01/2015,44562
206,11/01/2015,38112
205,18/01/2015,50355
204,25/01/2015,51922
203,01/02/2015,67496
202,08/02/2015,54229
201,15/02/2015,41548
200,22/02/2015,50872
199,01/03/2015,40578
198,08/03/2015,46025


1 predict 1

In [9]:
df1 = df_envi.copy()
df1["y"] = df1["Envi"].shift(-1)
df1.dropna(inplace=True)
df1["y"] = df1["y"].astype("int64")
df1.head(5)

,Date,Envi,y
207,04/01/2015,44562,38112
206,11/01/2015,38112,50355
205,18/01/2015,50355,51922
204,25/01/2015,51922,67496
203,01/02/2015,67496,54229


sequence predict 1

In [10]:
def createSequenceToOneDataSet(r, data:pd.DataFrame):  # Create Sequence DataSet
    df = data.copy()
    i = 1
    
    while i < r:
        df[f'x{i}'] = df['Envi'].shift(-i)
        i += 1

    if i == r:
        df['y'] = df['Envi'].shift(-i)

    df = df.dropna(axis=0)
    
    # change type of data
    for i in range(1, r):
        df[f'x{i}'] = df[f'x{i}'].astype("int64")
        
    df['y'] = df["y"].astype("int64")
    
    return df

df2 = createSequenceToOneDataSet(5, df_envi)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164 entries, 207 to 3338
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    164 non-null    object
 1   Envi    164 non-null    int64 
 2   x1      164 non-null    int64 
 3   x2      164 non-null    int64 
 4   x3      164 non-null    int64 
 5   x4      164 non-null    int64 
 6   y       164 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 10.2+ KB


- บันทึกไฟล์ csv ของแอปเปิ้ลสายพันธ์ุ Envi เพื่อใช้สำหรับสร้างโมเดลต่อไป

In [11]:
df1.to_csv("envi_1p1.csv", sep=",", encoding="utf-8", index=False)
df2.to_csv("envi_sp1.csv", sep=",", encoding="utf-8", index=False)